# Stock Prediction Model

## Dependencies

### Library Installation (if needed)

In [13]:
# Install libraries the first time
#! pip install -U yfinance pandas pathlib numpy 

### Importing Required Libraries

In [2]:
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import numpy as np
import pathlib as Path

## Data Preparation

In [15]:
symbols = pd.read_csv('../data/simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
symbols

array(['MATAS.CO', 'TRIFOR.CO', 'QQ.L', 'RNMBY', 'SAABF', 'BCKIY',
       'BAESY', 'IVSO.ST', 'NSKFF', 'GMAB', 'GN.CO', 'NVDA', 'LLY',
       'DANSKE.CO', 'CARL-B.CO', 'MAERSK-B.CO', 'RBREW.CO', 'ISS.CO',
       'DSV.CO', 'SCHO.CO', 'NETC.CO', 'JYSK.CO', 'ABBN.SW', 'TER',
       'PARKEN.CO', 'NFLX', 'TRMD-A.CO', 'STG.CO', 'NOVO-B.CO', 'EQNR',
       'NKT.CO', 'NSIS-B.CO', 'KCC.OL'], dtype=object)

#### Download and Process Historical Data

In [16]:
#symbols = ['AAPL']

In [12]:
yf.Ticker("AAPL").history(period='10y')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-04-06 00:00:00-04:00,27.797598,28.476514,27.766332,28.440781,148776000,0.0,0.0
2015-04-07 00:00:00-04:00,28.505541,28.612737,28.134818,28.141518,140049200,0.0,0.0
2015-04-08 00:00:00-04:00,28.105791,28.228622,27.909263,28.049959,149316800,0.0,0.0
2015-04-09 00:00:00-04:00,28.105785,28.268815,27.840026,28.264347,129936000,0.0,0.0
2015-04-10 00:00:00-04:00,28.128120,28.409513,27.974025,28.384947,160752000,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-31 00:00:00-04:00,217.009995,225.619995,216.229996,222.130005,65299300,0.0,0.0
2025-04-01 00:00:00-04:00,219.809998,223.679993,218.899994,223.190002,36412700,0.0,0.0
2025-04-02 00:00:00-04:00,221.320007,225.190002,221.020004,223.889999,35905900,0.0,0.0


In [ ]:
ticker = yf.Ticker('AAPL')
earning_dates = ticker.cashflow.columns.tolist()
cashflow = ticker.balance_sheet
cashflow_columns = cashflow[earning_dates[0]].keys().tolist()
pd.DataFrame(cashflow_columns)

,0
0,Free Cash Flow
1,Repurchase Of Capital Stock
2,Repayment Of Debt
3,Issuance Of Debt
4,Issuance Of Capital Stock
5,Capital Expenditure
6,Interest Paid Supplemental Data
7,Income Tax Paid Supplemental Data
8,End Cash Position
9,Beginning Cash Position


In [ ]:
df = pd.DataFrame()
for symbol in symbols:
    ticker = yf.Ticker(symbol)
    earning_dates = ticker.cash_flow.columns.tolist()
    cash_flow = ticker.cash_flow
    cash_flow_columns = cash_flow[earning_dates[0]].keys().tolist()
    balance_sheet = ticker.balance_sheet
    balance_sheet_columns = balance_sheet[earning_dates[0]].keys().tolist()
    income_statement = ticker.income_statement
    income_statement_columns = income_statement[earning_dates[0]].keys().tolist()

    for earning_date in earning_dates:
        row_data = {'Ticker': symbol, 'Date': earning_date}

        try:
            for column in cash_flow_columns:
                row_data[column] = cash_flow[earning_date][column]

            for column in balance_sheet_columns:
                row_data[column] = balance_sheet[earning_date][column]

            for column in income_statement_columns:
                row_data[column] = income_statement[earning_date][column]

            price_data = ticker.history(timeframe='10y')
        
        row_data['Close Price'] = price_data['Close'].iloc[-1]

        df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)

df.to_csv('../data/earnings_data.csv', index=False)

MATAS.CO...


$MATAS.CO: possibly delisted; no price data found  (1d 2024-03-30 -> 2024-04-01)
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)


TRIFOR.CO...


$TRIFOR.CO: possibly delisted; no price data found  (1d 2023-12-30 -> 2024-01-01)
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)
$TRIFOR.CO: possibly delisted; no price data found  (1d 2020-12-30 -> 2021-01-01) (Yahoo error = "Data doesn't exist for startDate = 1609282800, endDate = 1609455600")
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the releva

QQ.L...


...

$NKT.CO: possibly delisted; no price data found  (1d 2023-12-30 -> 2024-01-01)
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)


Error for NKT.CO on 2020-12-31 00:00:00: Timestamp('2020-12-31 00:00:00')
NSIS-B.CO...


$NSIS-B.CO: possibly delisted; no price data found  (1d 2023-12-30 -> 2024-01-01)
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)


KCC.OL...


$KCC.OL: possibly delisted; no price data found  (1d 2023-12-30 -> 2024-01-01)
C:\Users\Gamer\AppData\Local\Temp\ipykernel_3208\3458907841.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)


In [18]:
tickers = pd.read_csv(testFolder / 'filteredTickers.csv')['Ticker']
trainingTickers = np.random.choice(tickers, size=trainingSize, replace=False)
trainingRowAmount = len(pd.read_csv(testFolder / 'filteredTickers.csv'))

if getNewData:
    histData = pd.DataFrame()
    valid_tickers = []
    
    for ticker in trainingTickers:
        print(f"Processing {ticker}...")
        try:
            data = calculateFutureYearChange(ticker, timeFrame)
            if not data.empty:
                data['Ticker'] = ticker
                data['Industry'] = yf.Ticker(ticker).info.get('industry', 'Unknown')
                data['Date'] = pd.to_datetime(data['Date']).dt.tz_localize(None)
                
                # Enrich individual ticker data first
                ticker_data = enrichDataWithMetrics(data)
                histData = pd.concat([histData, ticker_data])
                
                # Check if metrics were added
                if 'ROIC' not in ticker_data.columns:
                    print(f"WARNING: Failed to add metrics for {ticker}")
                
                valid_tickers.append(ticker)
            else:
                print(f"Skipped {ticker} - insufficient data")
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    print(f"\nColumns in final dataset: {histData.columns.tolist()}")
    
    if not histData.empty:
        histData = enrichDataWithMetrics(histData)
        histData.to_csv(dataFolder / trainingData, index=True)
        # Verify no future targets leaked to past dates
        latest_date = pd.to_datetime(histData['Date']).max()
        if 'Future Year Change' in histData.columns:
            target_dates = histData[histData['Future Year Change'].notnull()]['Date']
            if any(pd.to_datetime(target_dates) > latest_date):
                raise ValueError("CRITICAL: Analyst targets contain future dates!")
        trainingRowAmount = len(histData)
        print(f"Saved training data with {trainingRowAmount} rows")
    else:
        print("Warning: No data collected - check your tickers list")

NameError: name 'testFolder' is not defined

### Short visualisation

In [ ]:
if getNewData:
    histData.head(5)

## Model Training

In [ ]:
if trainNewModel:
    TRAINING_CUTOFF = pd.to_datetime('2023-01-01')

    df = pd.read_csv(dataFolder / trainingData)
    dfCleaned = df.dropna(subset=['EV/EBIT', 'ROIC']).copy()

    # Convert 'Date' to datetime, parse UTC-aware dates, then make naive
    dfCleaned['Date'] = pd.to_datetime(dfCleaned['Date'], errors='coerce', utc=True).dt.tz_convert(None)
    dfCleaned = dfCleaned.dropna(subset=['Date'])

    # Clean 'EV/EBIT' and reset index
    dfCleaned['EV/EBIT'] = dfCleaned['EV/EBIT'].replace([np.inf, -np.inf], np.nan)
    dfCleaned = dfCleaned.dropna(subset=['EV/EBIT', 'ROIC'])
    dfCleaned = dfCleaned.reset_index(drop=True)
    dfCleaned.to_csv(dataFolder / trainingData, index=False)

    # Check for empty data
    if dfCleaned.empty:
        raise ValueError("The cleaned DataFrame is empty.")

    # Create splits with valid indices
    train_mask = dfCleaned['Date'] < TRAINING_CUTOFF
    valid_mask = ~train_mask
    splits = (list(dfCleaned[train_mask].index), list(dfCleaned[valid_mask].index))

    if not splits[0] or not splits[1]:
        raise ValueError("Empty training or validation split.")

    # Proceed with TabularPandas
    to = TabularPandas(
        dfCleaned, 
        procs=[Categorify, FillMissing, Normalize],
        y_names=yNames,
        cat_names=catNames, 
        cont_names=contNames,
        splits=splits
    )

    dls = to.dataloaders(bs=batchSize)

In [ ]:
if trainNewModel:
    learn = tabular_learner(dls, metrics=[rmse, mae])

    # Learning rate finder
    lr_find_results = learn.lr_find(suggest_funcs=(minimum, steep))

    # Debugging information
    print(f"Learning rate finder results: {lr_find_results}")

    # Extract learning rates
    lr_min, lr_steep = lr_find_results

    # Check if learning rates are valid
    if lr_min is None or lr_steep is None or lr_min == 0 or lr_steep == 0:
        raise ValueError("Learning rate finder did not return valid learning rates.")

    # Train
    print(f"Training for {epochs} epochs...")
    learn.fit_one_cycle(epochs, lr_max=lr_steep)
    print("Model training complete")

    learn.export(modelFolder / f'{modelName}.pkl')

## Testing and benchmarking

In [ ]:
def logEvaluation(model_name, mae, rmse, r2, model_folder, test_tickers):
    """Log evaluation metrics to CSV file"""
    log_file = model_folder / "modelEvaluations.csv"
    
    new_entry_df = pd.DataFrame([{
        "Model Name": model_name,
        "Timestamp": datetime.now().strftime('%Y-%m-%d %H:%M'),
        "MAE": f'{mae:.3f}',
        "RMSE": f'{rmse:.3f}',
        "R2": f'{r2:.3f}',
        "Epochs": epochs,
        "Training Size": trainingSize,
        "Training Rows": trainingRowAmount,
        "Test Size": len(test_tickers),
        "Cat Names": catNames,
        "Cont Names": contNames,
    }])
    
    try:
        log_df = pd.read_csv(log_file)
        log_df = pd.concat([log_df, new_entry_df], ignore_index=True)
    except FileNotFoundError:
        log_df = new_entry_df
        
    log_df.to_csv(log_file, index=False)
    print(f"Logged evaluation results to {log_file}")

def plotResults(results_df, model_name, model_folder):
    """Create and save visualization plots using all data points."""
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.scatter(results_df['Actual'], results_df['Predicted'], alpha=0.7, label='Predictions')
    min_val = min(results_df['Actual'].min(), results_df['Predicted'].min())
    max_val = max(results_df['Actual'].max(), results_df['Predicted'].max())
    plt.plot([min_val, max_val], [min_val, max_val], color='red', linestyle='--', label='Perfect Prediction')
    plt.title(f'Predicted vs. Actual Returns - {model_name}', fontsize=14)
    plt.xlabel('Actual Returns')
    plt.ylabel('Predicted Returns')
    plt.legend()
    plt.grid(alpha=0.5)
    
    plt.subplot(2, 1, 2)
    plt.scatter(results_df['Predicted'], results_df['Residual'], alpha=0.7)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.title('Residual Plot')
    plt.xlabel('Predicted Returns')
    plt.ylabel('Residual')
    plt.grid(alpha=0.5)
    
    plt.tight_layout()
    plt.show()

if trainNewModel:
    nonTrainingTickers = list(set(tickers) - set(trainingTickers))
    validTestData = []
    attempted_tickers = set()
    attempts = 0

    if testSize * 4 <= len(tickers):  
        max_attempts = testSize * 4 # Prevent infinite loops
    else:
        max_attempts = len(tickers)

    # Keep trying until we reach testSize or exhaust attempts
    while len(validTestData) < testSize and attempts < max_attempts:
        # Get a new ticker we haven't tried yet
        remaining_tickers = [t for t in nonTrainingTickers if t not in attempted_tickers]
        if not remaining_tickers:  # If all tried, reset attempted list
            attempted_tickers = set()
            remaining_tickers = nonTrainingTickers
            
        ticker = np.random.choice(remaining_tickers)
        attempted_tickers.add(ticker)
        attempts += 1

        # Fetch and validate data
        data = getTickerDataFrom1YrAgo(ticker)
        if not data.empty and not data[['EV/EBIT', 'ROIC']].isna().any().any():
            validTestData.append(data)

    if not validTestData:
        raise ValueError("No valid test data collected after multiple attempts")
        
    # Trim to exact testSize if we collected more
    validTestData = validTestData[:testSize]  
    combinedTestData = pd.concat(validTestData, ignore_index=True)

    # Clean data
    test_data_clean = combinedTestData.dropna(subset=['EV/EBIT', 'ROIC', 'Future Year Change'])
    
    if test_data_clean.empty:
        raise ValueError("No valid test data after cleaning NaN values")

    # Create test dataloader
    test_dl = learn.dls.test_dl(test_data_clean)
    preds, targs = learn.get_preds(dl=test_dl)
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'Predicted': preds.numpy().flatten(),
        'Actual': targs.numpy().flatten()
    })
    results_df['Residual'] = results_df['Actual'] - results_df['Predicted']

    # Calculate metrics
    mae = np.mean(np.abs(results_df['Residual']))
    rmse = np.sqrt(np.mean(results_df['Residual']**2))
    r2 = 1 - (np.sum(results_df['Residual']**2) / np.sum((results_df['Actual'] - results_df['Actual'].mean())**2))

    # Log and plot
    logEvaluation(modelName, mae, rmse, r2, modelFolder, test_data_clean['Ticker'].unique())
    plotResults(results_df, modelName, modelFolder)

    # Show collection stats
    print(f"Collected {len(validTestData)} valid test tickers (target: {testSize})")
    if attempts >= max_attempts:
        print(f"Warning: Reached max attempts ({max_attempts}). Some invalid tickers may remain.")  
    print(f"\nEvaluation Results:")
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²: {r2:.3f}")

## Predictions

In [ ]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
stockScreenerV1.0.pkl
stockScreenerV1.1.pkl
stockScreenerV1.10.pkl
stockScreenerV1.2.pkl
stockScreenerV1.3.pkl
stockScreenerV1.4.pkl
stockScreenerV1.5.pkl
stockScreenerV1.6.pkl
stockScreenerV1.7.pkl
stockScreenerV1.8.pkl
stockScreenerV1.9.pkl
stockScreenerV2.0.pkl
stockScreenerV2.1.pkl
stockScreenerV2.2.pkl


In [ ]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[0]
bestModel.head()

Model Name    stockScreenerV1.7
Timestamp      2025-01-27 08:45
MAE                       0.328
RMSE                      0.739
R2                        0.077
Name: 7, dtype: object

### Load model

In [ ]:
if os.name == 'nt':
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
else:
    pathlib.WindowsPath = pathlib.PosixPath

importedModel = Path(f"{bestModel['Model Name']}.pkl") # Change this if you want to try other models
learn = load_learner(modelFolder / importedModel)

In [ ]:
#predictionTarget = '95%'

In [ ]:
if predictionTarget != 'None':
    if predictionTarget == 'ALL':
        predictionTickers = tickers
    elif predictionTarget.endswith('%'):
        percentage = float(predictionTarget.strip('%')) / 100
        num_tickers = int(len(tickers) * percentage)
        predictionTickers = np.random.choice(tickers, size=num_tickers, replace=False).tolist()
    else:
        predictionTickers = [predictionTarget]

    # Fetch data for prediction tickers
    dfPrediction = pd.concat([getTickerData(ticker) for ticker in predictionTickers], ignore_index=True)

    # Ensure dfPrediction is a DataFrame
    if isinstance(dfPrediction, dict):
        dfPrediction = pd.DataFrame([dfPrediction])

    # Drop rows with NaN values in 'EV/EBIT' or 'ROIC'
    dfPrediction = dfPrediction.dropna(subset=['EV/EBIT', 'ROIC'])

    # Create test dataloader
    dl = learn.dls.test_dl(dfPrediction)
    dfPrediction.head()

    # Make predictions
    prediction = learn.get_preds(dl=dl)
    adr_df = pd.read_csv(testFolder / 'filteredTickers.csv')
    company_dict = dict(zip(adr_df['Ticker'], adr_df['Company']))

    if predictionTarget == 'ALL' or predictionTarget.endswith('%'):
        sorted_predictions = sorted(zip(predictionTickers, prediction[0]), key=lambda x: x[1], reverse=True)
        print(f"Got predictions for {len(sorted_predictions)} tickers, expected: {len(predictionTickers)}")
        print(f"Prediction for best performing tickers:")
        for symbol, pred in sorted_predictions:
            company_name = company_dict.get(symbol, 'Unknown')
            print(f"{symbol} ({company_name}): {pred[0].item() * 100:.2f}%")
    else:
        company_name = company_dict.get(predictionTarget, 'Unknown')
        print(f"Prediction for {predictionTarget} ({company_name}):")
        print(f"{prediction[0][0][0].item() * 100:.2f}%")
    print("Free money?!")

NameError: name 'getTickerData' is not defined